Link: https://www.kaggle.com/datasets/spscientist/students-performance-in-exams?resource=download

In [15]:
import numpy as np
import pandas as pd

df = pd.read_csv('DataSets\stud_perf_math_reading_writing.csv')
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


turn categorical attributes into numerical attributes


In [16]:
df = df[[c for c in df if c not in ['math score']] + ['math score']]

ord_education = ['some high school', 'high school', 'some college', "associate's degree", "bachelor's degree", "master's degree"]
for num, level in enumerate(ord_education):
    df.replace(to_replace=level, value=num, inplace=True)

feats_to_one_hot_encode = ['gender', 'race/ethnicity', 'lunch', 'test preparation course']
df = pd.get_dummies(df, prefix=feats_to_one_hot_encode)

feats_to_standardize = ['parental level of education', 'reading score', 'writing score']
for feat in feats_to_standardize:
    df[feat] = (df[feat] - df[feat].mean()) / df[feat].std()

# features = df.iloc[:, :df.shape[1] - 1].values
# labels = df.loc[:, 'math score'].values
clone = df
clone = clone.drop('math score', axis=1)
features = clone.iloc[:].values
labels = df.loc[:, 'math score'].values

In [46]:
clone

,parental level of education,reading score,writing score,gender_female,gender_male,race/ethnicity_group A,race/ethnicity_group B,race/ethnicity_group C,race/ethnicity_group D,race/ethnicity_group E,lunch_free/reduced,lunch_standard,test preparation course_completed,test preparation course_none
0,1.314084,0.193902,0.391296,1,0,0,1,0,0,0,0,1,0,1
1,-0.055467,1.426762,1.312612,1,0,0,0,1,0,0,0,1,1,0
2,1.998860,1.769223,1.641653,1,0,0,1,0,0,0,0,1,0,1
3,0.629309,-0.833482,-1.582952,0,1,1,0,0,0,0,1,0,0,1
4,-0.055467,0.604855,0.457104,0,1,0,0,1,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1.998860,2.043192,1.773270,1,0,0,0,0,0,1,0,1,1,0
996,-0.740242,-0.970467,-0.859061,0,1,0,0,1,0,0,1,0,0,1
997,-0.740242,0.125409,-0.200978,1,0,0,0,1,0,0,1,0,1,0
998,-0.055467,0.604855,0.588721,1,0,0,0,0,1,0,0,1,1,0


In [38]:
features[0]

array([1.31408412, 0.19390156, 0.39129601, 1.        , 0.        ,
       0.        , 1.        , 0.        , 0.        , 0.        ,
       0.        , 1.        , 0.        , 1.        ])

In [17]:
def generateRandom(data, features, labels):
  perm_idx = np.random.permutation(data.shape[0])
  vali_num = int(data.shape[0] * 0.2)
  vali_idx = perm_idx[:vali_num]
  train_idx = perm_idx[vali_num:]
  train_features = features[train_idx, :]
  train_labels = labels[train_idx]
  vali_features = features[vali_idx, :]
  vali_labels = labels[vali_idx]
  return train_features, train_labels, vali_features, vali_labels

train_features, train_labels, vali_features, vali_labels = generateRandom(df, features, labels)
print(train_features.shape)
print(train_labels.shape)
print(vali_features.shape)
print(vali_labels.shape)

(800, 14)
(800,)
(200, 14)
(200,)


In [40]:
train_features[1]

array([ 0.62930865, -0.01157519,  0.12806291,  1.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
        0.        ,  1.        ,  0.        ,  1.        ])

In [18]:
def KNN(train_features, train_labels, test_features, k=10):
    vali_pred = []
    for i in range(test_features.shape[0]):
        x = test_features[i, :]  
        distances = np.sqrt(np.sum((x - train_features) ** 2, axis=1))
        topk_idx = np.argpartition(distances, k)[:k]
        topk_labels = list(train_labels[topk_idx])
        pred = max(topk_labels, key=topk_labels.count)
        vali_pred.append(pred)
    return np.array(vali_pred)
for k_tuner in range(5, 16):
    pred = KNN(train_features, train_labels, vali_features, k=k_tuner)

    count = 0
    margin = 7
    for i in range(pred.size):
        if vali_labels[i] - margin <= pred[i] and pred[i] <= vali_labels[i] + margin:
            count += 1

    print("K: {}, Accuracy: {}".format(k_tuner, str(count / vali_features.shape[0])))

K: 5, Accuracy: 0.52
K: 6, Accuracy: 0.54
K: 7, Accuracy: 0.495
K: 8, Accuracy: 0.505
K: 9, Accuracy: 0.52
K: 10, Accuracy: 0.535
K: 11, Accuracy: 0.545
K: 12, Accuracy: 0.56
K: 13, Accuracy: 0.585
K: 14, Accuracy: 0.59
K: 15, Accuracy: 0.565


In [19]:
# parental level of education - index 0
# reading score- index 1
# writing score- index 2
# gender_female- index 3
# gender_male- index 4
# race/ethnicity_group A- index 5
# race/ethnicity_group B- index 6
# race/ethnicity_group C- index 7
# race/ethnicity_group D- index 8
# race/ethnicity_group E- index 9
# lunch_free/reduced- index 10
# lunch_standard- index 11
# course_completed- index 12
# course_not_completed- index 13

testSample = [[ 1.,  0.12540931,  0.78614567,  0.  , 1. ,  0.,  0.,  0.,  1.,0.,  1.,  0.,  1., 0.], #Parents influence
              [ 0.,  0.12540931,  0.78614567,  0.  , 1. ,  0.,  0.,  0.,  1.,0.,  1.,  0.,  1., 0.],
              [ -1.,  0.12540931,  0.78614567,  0.  , 1. ,  0.,  0.,  0.,  1.,0.,  1.,  0.,  1., 0.],
              [ 1.,  0.12540931,  0.78614567,  0.  , 1. ,  0.,  0.,  0.,  1.,0.,  1.,  0.,  1., 0.], #Course Complete 
              [ 1.,  0.12540931,  0.78614567,  0.  , 1. ,  0.,  0.,  0.,  1.,0.,  1.,  0.,  0., 1.], 
              [ 1.,  -1,  0.78614567,  0.  , 1. ,  0.,  0.,  0.,  1.,0.,  1.,  0.,  0., 1.], #Reading score  
              [ 1.,  0,  0.78614567,  0.  , 1. ,  0.,  0.,  0.,  1.,0.,  1.,  0.,  0., 1.],  
              [ 1.,  1,  0.78614567,  0.  , 1. ,  0.,  0.,  0.,  1.,0.,  1.,  0.,  0., 1.],
              [ 1.,  2,  0.78614567,  0.  , 1. ,  0.,  0.,  0.,  1.,0.,  1.,  0.,  0., 1.]]


In [20]:
pred = KNN(train_features, train_labels, np.array(testSample), k=6)
print("Highly Educated Parent :", pred[0])
print("Educated Parent :", pred[1])
print("Uneducated Parent :", pred[2])
print("Course Completed :", pred[3])
print("Course Not Completed :", pred[4])
print("Bad Reading Score :", pred[5])
print("Ok Reading Score :", pred[6])
print("Average Reading Score :", pred[7])
print("Good Reading Score :", pred[8])

Highly Educated Parent : 79
Educated Parent : 74
Uneducated Parent : 78
Course Completed : 79
Course Not Completed : 75
Bad Reading Score : 63
Ok Reading Score : 75
Average Reading Score : 63
Good Reading Score : 90
